Q1. Install MLflow
To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with pip or conda.

Once you installed the package, run the command mlflow --version and check the output.

What's the version that you have? A: I have the 2.22.0 mlflow version 

In [42]:
import mlflow

In [43]:
mlflow.__version__#my version is 2.22.0

'2.22.0'

Q2. Download and preprocess the data
We'll use the Green Taxi Trip Records dataset to predict the duration of each trip.

Download the data for January, February and March 2023 in parquet format from here.

Use the script preprocess_data.py located in the folder homework to preprocess the data.

The script will:

load the data from the folder <TAXI_DATA_FOLDER> (the folder where you have downloaded the data),
fit a DictVectorizer on the training set (January 2023 data),
save the preprocessed datasets and the DictVectorizer to disk.
Your task is to download the datasets and then execute this command:

python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
Tip: go to 02-experiment-tracking/homework/ folder before executing the command and change the value of <TAXI_DATA_FOLDER> to the location where you saved the data.

How many files were saved to OUTPUT_FOLDER? A: The output folder contain 4 files: dv.pkl, test.pkl, train.pkl, val.pkl

In [44]:
import os
os.chdir(".")
os.listdir()

['artifacts_local',
 'backend.db',
 'data',
 'homework.ipynb',
 'hpo.py',
 'mlruns',
 'output',
 'preprocess_data.py',
 'register_model.py',
 'train.py',
 '__pycache__']

In [45]:
!python preprocess_data.py --raw_data_path data/ --dest_path ./output

Q3. Train a model with autolog
We will train a RandomForestRegressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script train.py for this exercise, which can be also found in the folder homework.

The script will:

load the datasets produced by the previous step,
train the model on the training set,
calculate the RMSE score on the validation set.
Your task is to modify the script to enable autologging with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

Tip 1: don't forget to wrap the training code with a with mlflow.start_run(): statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the min_samples_split parameter: A: The value of the min_samples_split parameter is 2

In [46]:
!python train.py --data_path output/ 

Q4. Launch the tracking server locally
Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to:

launch the tracking server on your local machine,
select a SQLite db for the backend store and a folder called artifacts for the artifacts store.
You should keep the tracking server running to work on the next two exercises that use the server.

In addition to backend-store-uri, what else do you need to pass to properly configure the server?

default-artifact-root
serve-artifacts
artifacts-only
artifacts-destination

A: We will need default-artifact-root, for example: 
mlflow server --backend-store-uri sqlite:///backend.db --defautl-artifact-root ./artifacts_local

Q5. Tune model hyperparameters
Now let's try to reduce the validation error by tuning the hyperparameters of the RandomForestRegressor using hyperopt. We have prepared the script hpo.py for this exercise.

Your task is to modify the script hpo.py and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the objective function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called random-forest-hyperopt to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

the list of hyperparameters that are passed to the objective function during the optimization,
the RMSE obtained on the validation set (February 2023 data).
What's the best validation RMSE that you got? A: I got 5.335 the best validation RMSE

4.817
5.335
5.818
6.336

In [52]:
!python hpo.py --data_path output/ 


  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]
                                                      
🏃 View run skittish-stag-839 at: http://127.0.0.1:5000/#/experiments/1/runs/7ab5470e445343948851a77dd1b74651


  0%|          | 0/15 [00:10<?, ?trial/s, best loss=?]
                                                      
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


  7%|▋         | 1/15 [00:10<02:21, 10.13s/trial, best loss: 5.370086069268862]
                                                                               
🏃 View run awesome-ox-321 at: http://127.0.0.1:5000/#/experiments/1/runs/62eeccfbf6024411a2f79461f6767035


  7%|▋         | 1/15 [00:10<02:21, 10.13s/trial, best loss: 5.370086069268862]
                                                                               
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


 13%|█▎        | 2/15 [00:10<00:58,  4.50s/trial, best loss: 5.370086069268862]
                                    

Q6. Promote the best model to the model registry
The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called register_model.py, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called random-forest-best-models.

Your task is to update the script register_model.py so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method search_runs from the MlflowClient to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method mlflow.register_model and you will need to pass the right model_uri in the form of a string that looks like this: "runs:/<RUN_ID>/model", and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model? A: My best model has a test RMSE value: 5.567

In [54]:
!python register_model.py --data_path output/ 

🏃 View run bold-kit-738 at: http://127.0.0.1:5000/#/experiments/2/runs/1eccbf84c4dd45f5a961198647b6a634
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run delicate-steed-196 at: http://127.0.0.1:5000/#/experiments/2/runs/44d4a6f0f3054734b04286cf7772d1a8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run intrigued-boar-976 at: http://127.0.0.1:5000/#/experiments/2/runs/9148db79eccd44099adc31f437211bc7
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run funny-fawn-551 at: http://127.0.0.1:5000/#/experiments/2/runs/b14397b027f5472ebc14f7c9d0e475ef
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run luminous-rook-509 at: http://127.0.0.1:5000/#/experiments/2/runs/5cbc8ff9614440509931196fe8edde55
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


Registered model 'nyc-taxi-best-run' already exists. Creating a new version of this model...
2025/05/23 21:44:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-best-run, version 1
Created version '1' of model 'nyc-taxi-best-run'.
